In [1]:
import pandas as pd
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import tensorflow as tf
import tensorflow_datasets as tfds
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import os

2025-04-08 10:25:27.106948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744122327.120804 1518666 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744122327.124838 1518666 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744122327.135986 1518666 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744122327.136011 1518666 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744122327.136012 1518666 computation_placer.cc:177] computation placer alr

#### Melanoma Classifier

In [2]:
os.getcwd()

'/home/en520-ikarhul1'

In [3]:
dir = '/scratch4/en520-lmorove1/en520-ikarhul1/project'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.ImageFolder(root=dir+'/melanoma_cancer_dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root=dir+'/melanoma_cancer_dataset/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=2)

images, labels = next(iter(train_loader))

print(images.shape)
print(labels.shape)

torch.Size([32, 3, 64, 64])
torch.Size([32])


In [6]:
class MelanomaClassifier(nn.Module):
    def __init__(self, num_classes=1):
        super(MelanomaClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        # 64x64 input -> after 3 pooling layers: 64/2/2/2 = 8
        self.classifier = nn.Sequential(
            nn.Linear(128 * 8 * 8, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [20]:
model = MelanomaClassifier()
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10
batch_size = 32

losses = []

for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.unsqueeze(1).float().to(device)
        pred = model(data)
        loss = criterion(pred, targets)

        optim.zero_grad()
        loss.backward()
        optim.step()

    losses.append(loss.item())
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 1/10, Loss: 0.0606
Epoch 2/10, Loss: 0.0340
Epoch 3/10, Loss: 0.0646
Epoch 4/10, Loss: 0.2409
Epoch 5/10, Loss: 0.1053
Epoch 6/10, Loss: 0.0341
Epoch 7/10, Loss: 0.0510
Epoch 8/10, Loss: 0.2161
Epoch 9/10, Loss: 0.2850
Epoch 10/10, Loss: 0.1137


In [22]:
def evaluate_model(model, test_loader):
    model.eval()
    
    all_preds = []
    all_probs = []
    all_targets = []

    with torch.no_grad():
        for data, targets in test_loader:  
            outputs = model(data)
            probs = torch.sigmoid(outputs).squeeze()
            preds = (probs > 0.5).float()
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    all_preds = np.array(all_preds)
    all_probs = np.array(all_probs)
    all_targets = np.array(all_targets)

    accuracy = accuracy_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds, zero_division=0)
    recall = recall_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds)

    try:
        auc = roc_auc_score(all_targets, all_probs)
    except:
        auc = 0.0

    cm = confusion_matrix(all_targets, all_preds)
    
    return {'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'auc': auc,
            'confusion_matrix': cm,
            'predictions': all_preds,
            'probabilities': all_probs,
            'targets': all_targets}

In [21]:
results = evaluate_model(model, test_loader)

print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1 Score: {results['f1_score']:.4f}")
print(f"AUC: {results['auc']:.4f}")

Accuracy: 0.9150
Precision: 0.9406
Recall: 0.8860
F1 Score: 0.9125
AUC: 0.9713


#### EfficientNet

In [8]:
import torchvision
print(torchvision.__version__)

0.20.1


In [9]:
train_dataset = datasets.ImageFolder(root=dir+'/melanoma_cancer_dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root=dir+'/melanoma_cancer_dataset/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=2)

In [14]:
model = efficientnet_b0(weights=None)
state_dict = torch.load(dir+'/efficientnet_b0_rwightman-3dd342df.pth', map_location='cpu', weights_only=True)
model.load_state_dict(state_dict)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 1)
model = model.to(device)

In [15]:
criterion = nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10
losses = []

for epoch in range(epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.unsqueeze(1).float().to(device)
        pred = model(data)
        loss = criterion(pred, targets)

        optim.zero_grad()
        loss.backward()
        optim.step()

    losses.append(loss.item())
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 1/10, Loss: 0.1908
Epoch 2/10, Loss: 0.9254
Epoch 3/10, Loss: 0.0427
Epoch 4/10, Loss: 0.0745
Epoch 5/10, Loss: 0.0447
Epoch 6/10, Loss: 0.8541
Epoch 7/10, Loss: 1.6314
Epoch 8/10, Loss: 0.3684
Epoch 9/10, Loss: 0.1747
Epoch 10/10, Loss: 0.1165


In [19]:
results = evaluate_model(model, test_loader)

print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1 Score: {results['f1_score']:.4f}")
print(f"AUC: {results['auc']:.4f}")

Accuracy: 0.9070
Precision: 0.9231
Recall: 0.8880
F1 Score: 0.9052
AUC: 0.9717
